In [1]:
import torch
from supervoice.tokenizer import Tokenizer
from supervoice.model import SupervoiceGPT
from train_config import config

In [2]:
# Model
tokenizer = Tokenizer(config, "tokenizer.model")
model = SupervoiceGPT(config)
checkpoint = torch.load(f'./output/gpt_first.pt', map_location="cpu")
model.load_state_dict(checkpoint['model'])
model.eval()

RuntimeError: Error(s) in loading state_dict for SupervoiceGPT:
	size mismatch for input_embedding.weight: copying a param with shape torch.Size([916, 512]) from checkpoint, the shape in current model is torch.Size([16384, 512]).
	size mismatch for prediction_head.weight: copying a param with shape torch.Size([916, 512]) from checkpoint, the shape in current model is torch.Size([16384, 512]).

In [ ]:
# Prepare input
# "He was anxious to make this clear."
# ç iː  w ə z  æ ɲ c ʃ ə s  t ə  m ej k  d̪ ɪ s  c ʎ ɪ ɹ
input = "hey, you, what time is it?".lower()
expected = ""
tokenized = tokenizer(input).tolist()
tokenized = [tokenizer.sequence_begin_token_id, tokenizer.text_begin_token_id] + tokenized + [tokenizer.text_end_token_id, tokenizer.phonemes_begin_token_id]
tokenized = torch.tensor(tokenized)
stop_tokens = [tokenizer.sequence_end_token_id]
max_generate_len = 512

In [ ]:
# Generate
input = tokenized
res = model.generate(input.unsqueeze(0), max_new_tokens = max_generate_len, stop_tokens = stop_tokens, top_k = 6).squeeze(0)
res = res.tolist()
print(res)
print(tokenizer.decode(res))

In [ ]:
# Generate
# input = torch.tensor([tokenizer.phonemes_begin_token_id])
input = tokenized
total = 16
probs, indices = model.predict_next(input, total, trim_generated = False)
probs = probs.tolist()
tokens = tokenizer.reverse(indices.tolist())
for i in range(total):
    print(tokens[i] + ": " + str(probs[i]))

In [ ]:
# Iterative
input = tokenized

In [ ]:
total = 16
probs, indices = model.predict_next(input, total, trim_generated = False)
probs = probs.tolist()
tokens = tokenizer.reverse(indices.tolist())
for i in range(total):
    print(tokens[i] + ": " + str(probs[i]))
input = torch.tensor(input.tolist() + [indices[0]]).long()